***GENERATED CODE FOR apptest3 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '9', 'mean': '', 'stddev': '', 'min': 'AURORA', 'max': 'DENVER', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Category'}, {'feature_label': 'Product_Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Sub-Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Sub-Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'Appliances', 'max': 'Paper', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Sub-Category'}, {'feature_label': 'Product_Sub-Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Sub-Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Small Business', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Segment'}, {'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'First Name', 'transformation_label': 'String Indexer'}], 'feature': 'First Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'ANXXXX', 'max': 'TEXXX', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'First Name'}, {'feature_label': 'First Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('First Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Last Name', 'transformation_label': 'String Indexer'}], 'feature': 'Last Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'BRXX', 'max': 'WEXXXXXXXXX', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Last Name'}, {'feature_label': 'Last Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Last Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Address', 'transformation_label': 'String Indexer'}], 'feature': 'Address', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': '1030 S DOVER ST', 'max': '431 MARION ST', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Address'}, {'feature_label': 'Address', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Address')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'CO', 'max': 'CO', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MOSAIC HOUSEHOLD', 'transformation_label': 'String Indexer'}], 'feature': 'MOSAIC HOUSEHOLD', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'C11', 'max': 'J34', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC HOUSEHOLD'}, {'feature_label': 'MOSAIC HOUSEHOLD', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MOSAIC HOUSEHOLD')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MOSAIC DESCRIPTION', 'transformation_label': 'String Indexer'}], 'feature': 'MOSAIC DESCRIPTION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'Autumn Years', 'max': 'Thriving Boomers', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC DESCRIPTION'}, {'feature_label': 'MOSAIC DESCRIPTION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MOSAIC DESCRIPTION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Channel', 'transformation_label': 'String Indexer'}], 'feature': 'Channel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9', 'mean': '', 'stddev': '', 'min': 'Store', 'max': 'eCommerce', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Channel'}, {'feature_label': 'Channel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Channel')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run apptest3Hooks.ipynb
try:
	#sourcePreExecutionHook()

	apptesthdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1674827742.csv', 'filename': 'tempFile1674827742.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(apptesthdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run apptest3Hooks.ipynb
try:
	#transformationPreExecutionHook()

	apptestautofe = TransformationMain.run(apptesthdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "AURORA", "max": "DENVER", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Product_Category", "transformation_label": "String Indexer"}], "feature": "Product_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Category"}, {"transformationsData": [{"feature_label": "Product_Sub-Category", "transformation_label": "String Indexer"}], "feature": "Product_Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "Appliances", "max": "Paper", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Sub-Category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "1.11", "stddev": "0.33", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "1180.89", "stddev": "23.49", "min": "1146", "max": "1206", "missing": "0"}, "updatedLabel": "Customer ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store Number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "100.0", "stddev": "0.0", "min": "100", "max": "100", "missing": "0"}, "updatedLabel": "Store Number"}, {"transformationsData": [{"feature_label": "Customer Segment", "transformation_label": "String Indexer"}], "feature": "Customer Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "Consumer", "max": "Small Business", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Segment"}, {"transformationsData": [{"feature_label": "First Name", "transformation_label": "String Indexer"}], "feature": "First Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "ANXXXX", "max": "TEXXX", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "First Name"}, {"transformationsData": [{"feature_label": "Last Name", "transformation_label": "String Indexer"}], "feature": "Last Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "BRXX", "max": "WEXXXXXXXXX", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Last Name"}, {"transformationsData": [{"feature_label": "Address", "transformation_label": "String Indexer"}], "feature": "Address", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "1030 S DOVER ST", "max": "431 MARION ST", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Address"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "CO", "max": "CO", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "80151.44", "stddev": "103.88", "min": "80013", "max": "80226", "missing": "0"}, "updatedLabel": "Zip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DriveTime", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9", "mean": "19.95", "stddev": "7.63", "min": "11.56", "max": "27.59", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DriveTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length of Residense", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "18.0", "stddev": "11.12", "min": "7", "max": "36", "missing": "0"}, "updatedLabel": "Length of Residense"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MOR BANK: UPSCALE MERCHANDISE BUYER", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "MOR BANK: UPSCALE MERCHAN..."}, {"transformationsData": [{"feature_label": "MOSAIC HOUSEHOLD", "transformation_label": "String Indexer"}], "feature": "MOSAIC HOUSEHOLD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "C11", "max": "J34", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC HOUSEHOLD"}, {"transformationsData": [{"feature_label": "MOSAIC DESCRIPTION", "transformation_label": "String Indexer"}], "feature": "MOSAIC DESCRIPTION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "Autumn Years", "max": "Thriving Boomers", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC DESCRIPTION"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9", "mean": "-104.93", "stddev": "0.14", "min": "-105.093008", "max": "-104.758496", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9", "mean": "39.7", "stddev": "0.04", "min": "39.644417", "max": "39.737073", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9", "mean": "100.0", "stddev": "0.0", "min": "100", "max": "100", "missing": "0"}, "updatedLabel": "Store ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9", "mean": "-104.91", "stddev": "0.0", "min": "-104.905387", "max": "-104.905387", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9", "mean": "39.7", "stddev": "0.0", "min": "39.699065", "max": "39.699065", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lat"}, {"transformationsData": [{"feature_label": "Channel", "transformation_label": "String Indexer"}], "feature": "Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9", "mean": "", "stddev": "", "min": "Store", "max": "eCommerce", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Channel"}]}))

	#transformationPostExecutionHook(apptestautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run apptest3Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(apptestautofe, ["Count", "Customer ID", "Store Number", "Zip", "DriveTime", "Length of Residense", "MOR BANK: UPSCALE MERCHANDISE BUYER", "Customer_Lon", "Customer_Lat", "Store ID", "Store_Lon", "Store_Lat", "City_stringindexer", "Product_Category_stringindexer", "Product_Sub-Category_stringindexer", "Customer Segment_stringindexer", "First Name_stringindexer", "Last Name_stringindexer", "Address_stringindexer", "State_stringindexer", "MOSAIC HOUSEHOLD_stringindexer", "MOSAIC DESCRIPTION_stringindexer"], "Channel_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

